In [1]:
import numpy as np
import itertools
import cv2
import random
import copy
from time import sleep, time
import pickle

### Parâmetros

In [2]:
def initialize_parameters(new):
    params = {}
    params['nlayer'] = 2
    params['wlen'] = 3
    params['wsize'] = params['wlen'] ** 2
    params['dim'] = 56
    params['train_size'] = 30
    params['val_size'] = 10
    params['test_size'] = 10

    if new:
        Wini = np.array([np.nan, 1., np.nan, 1., 1., 1., np.nan, 1., np.nan])
        params['W'] = [Wini.copy() for _ in range(params['nlayer'])]
        params['joint'] = [create_joint(Wini) for _ in range(params['nlayer'])]
    else:
        params['joint'] = np.load('joint.txt', allow_pickle=True)
        params['W'] = np.load('W.txt', allow_pickle=True)

    params['windows_continuos'] = np.load('windows_continuos.txt', allow_pickle=True)

    return params

In [65]:
def create_joint(W):
    Ji=[]
    ni = int(W[~np.isnan(W)].sum())
    for i in itertools.product([0, 1], repeat=ni):
        Ji.append(''.join(np.array(i).astype(str)))
    np.random.seed(0)
    return np.c_[Ji, np.random.randint(2, size=len(Ji))]

In [66]:
params = initialize_parameters(new = True)

In [126]:
nlayer = params['nlayer']
wlen = params['wlen']
wsize = params['wsize']
dim = params['dim']
train_size = params['train_size']
val_size = params['val_size']
test_size = params['test_size']
W = params['W']
joint = params['joint']
windows_continuos = params['windows_continuos']

In [301]:
W_test =[np.array([np.nan, np.nan, np.nan, np.nan, 1., 1., np.nan,np.nan, np.nan])]

In [340]:
joint_test = [np.array([['00', '0'],
       ['01', '1'],
       ['10', '0'],
       ['11', '1']], dtype='<U11')]
joint_test

[array([['00', '0'],
        ['01', '1'],
        ['10', '0'],
        ['11', '1']], dtype='<U11')]

In [341]:
joint_test[0]

array([['00', '0'],
       ['01', '1'],
       ['10', '0'],
       ['11', '1']], dtype='<U11')

In [342]:
x_test = np.array([
    [1., 1., 0],
    [1., 1., 0],
    [0, 0, 1.]
])
x_test

array([[1., 1., 0.],
       [1., 1., 0.],
       [0., 0., 1.]])

In [343]:
apply_window_(x_test, W_test[0], joint_test[0])

array([[1, 0, 0],
       [1, 0, 0],
       [0, 1, 0]])

In [344]:
x2_test = copy.deepcopy(x_test)
x2_test[x2_test==0]=-1
x2_test

array([[ 1.,  1., -1.],
       [ 1.,  1., -1.],
       [-1., -1.,  1.]])

In [345]:
find_minterms(joint_test[0])

['01', '11']

In [346]:
nlayer=1

In [347]:
def create_w_matrices( W, joint):
        matrices = []
        for k in range(nlayer):
            matrix_k = []
            reduced_minterms = find_minterms(joint[k])
            for minterm in range(len(reduced_minterms)):
                matrix = np.copy(W[k])
                c=0
                for i in np.where(W[k]==1)[0]:
                    matrix[i] = reduced_minterms[minterm][c]
                    c+=1
                matrix = np.where(matrix==0,-1,matrix)
                matrix = np.nan_to_num(matrix)
                matrix_k.append(matrix.reshape((3, 3)))
            matrices.append(matrix_k)
        return matrices

In [348]:
W_matrices = create_w_matrices(W_test, joint_test)
W_matrices

[[array([[ 0.,  0.,  0.],
         [ 0., -1.,  1.],
         [ 0.,  0.,  0.]]),
  array([[0., 0., 0.],
         [0., 1., 1.],
         [0., 0., 0.]])]]

In [349]:
W_matrices[0][0]

array([[ 0.,  0.,  0.],
       [ 0., -1.,  1.],
       [ 0.,  0.,  0.]])

In [350]:
bias = np.nansum(W_test, axis=1) - 1
bias

array([1.])

In [351]:
bias[0]

1.0

In [370]:
increase = 1

In [376]:
def apply_convolve(img, W_matrices, bias):
        img_c = np.zeros([img.shape[0], img.shape[1]], dtype=float)
        for kernel in W_matrices: 
            img_b = cv2.copyMakeBorder(img, 1, 1, 1, 1, cv2.BORDER_CONSTANT, None, value = -1) 
            img_r = cv2.filter2D(img_b, -1, kernel)
            img_r = img_r-bias
            img_r = (img_r > 0).astype(float)
            img_c += img_r#[increase:img_r.shape[0]-increase, increase:img_r.shape[1]-increase]
            
        img_c[img_c == 0] = -1
        return img_c

In [377]:
rr = apply_convolve(x2_test, W_matrices[0], bias[0])
rr

ValueError: operands could not be broadcast together with shapes (3,3) (5,5) (3,3) 

In [369]:
rr.shape[1]

5

In [385]:
error_ep = {"error_train":[],"error_val":[],"W":[],"joint":[]}


In [381]:
error_ep["error_train"].append(1)

In [386]:
if error_ep["error_train"]:
    print('não vazio')
else:
    print('vazio')

vazio


In [388]:
np.nan<0

False

In [396]:
len(random_numbers)
n_ep

5000408

In [400]:
random.seed(0)
n_ep = 10000
random_list = random.sample(range(0, 1000000000), n_ep)
random_list[:50]

[906691059,
 413653999,
 813847339,
 955892128,
 451585301,
 43469773,
 278009742,
 548977048,
 521760889,
 434794718,
 985946604,
 841597326,
 891047768,
 325679554,
 511742081,
 384452587,
 626401695,
 957413342,
 975078788,
 234551094,
 541903389,
 149544006,
 302621084,
 150050891,
 811538590,
 101823753,
 663968655,
 858351976,
 268979133,
 976832602,
 571835844,
 757172936,
 869964135,
 646287425,
 968693314,
 157798602,
 333018422,
 106046331,
 783650878,
 79180332,
 965120263,
 913189317,
 734422154,
 354546567,
 506959381,
 601095367,
 108127101,
 379880545,
 466188456,
 339513621]

In [406]:
count = 0

In [407]:
W = np.array([np.nan, np.nan, np.nan, np.nan, 1., 1., np.nan, np.nan, np.nan])

In [410]:
Ji=[]
ni = int(W[~np.isnan(W)].sum())
for i in itertools.product([0, 1], repeat=ni):
    Ji.append(''.join(np.array(i).astype(str)))
random.seed(random_list[count])
count +=1
joint = np.c_[Ji, np.random.randint(2, size=len(Ji))]
joint

array([['00', '1'],
       ['01', '1'],
       ['10', '1'],
       ['11', '1']], dtype='<U11')

In [411]:
Ji

['00', '01', '10', '11']

In [427]:
random.sample(range(0, 10), 2)

[1, 8]

In [441]:
np.random.randint(0, 10, 5)

array([9, 0, 4, 7, 3])

In [420]:
np.random.seed(0)
np.random.randint(2, size=len(Ji))

array([0, 1, 1, 0])

In [426]:
random.seed(random_list[count])
ix = random.sample(range(10), 4)
ix

[2, 4, 0, 6]

In [ ]:
if n<len(v):
    random.seed(self.random_list[self.count])
    ix = random.sample(range(len(v)), n)
    self.count+=1
    else:
    ix = range(len(v))
ix

In [ ]:
train = []
ytrain = []
for img in range(1,train_size+1):
    x = cv2.imread(f'./x/train{img:02d}.jpg', cv2.IMREAD_GRAYSCALE)
    y = cv2.imread(f'./y/train{img:02d}.jpg', cv2.IMREAD_GRAYSCALE)
    train.append(convert_binary(x))
    ytrain.append(convert_binary(y))

In [ ]:
train = []
ytrain = []
for img in range(1,train_size+1):
    x = cv2.imread(f'./x/train{img:02d}.jpg', cv2.IMREAD_GRAYSCALE)
    y = cv2.imread(f'./y/train{img:02d}.jpg', cv2.IMREAD_GRAYSCALE)
    train.append(convert_binary(x))
    ytrain.append(convert_binary(y))

In [392]:
random.seed(0)
random.randint(0, len(windows_continuos)-1)

98

In [328]:
cv2.filter2D(x2_test, -1, W_matrices[0][0])-1

array([[ 2.,  2., -2.],
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.]])

In [333]:
W_matrices[0][0].reshape((3, 3))

array([[0., 0., 0.],
       [0., 1., 1.],
       [0., 0., 0.]])

In [356]:
from scipy.signal import convolve2d

rr = convolve2d(x2_test, W_matrices[0][0].reshape((3, 3)), mode='full', boundary='fill', fillvalue=-1)-1
rr[1:4, 1:4]


array([[-3., -1.,  1.],
       [-3., -1.,  1.],
       [-1., -1., -3.]])

In [127]:
W

[array([nan,  1., nan,  1.,  1.,  1., nan,  1., nan]),
 array([nan,  1., nan,  1.,  1.,  1., nan,  1., nan])]

In [118]:

def find_minterms(truth_table):
    inputs = truth_table[:, 0]
    results = truth_table[:, 1]
    minterms = []
    for i in np.where(results =='1')[0]:
        minterms.append(inputs[i])
    minterms

    return minterms


In [287]:
def create_w_matrices(W, joint):
    matrices = []
    for k in range(nlayer):
        matrix_k = []
        reduced_minterms = find_minterms(joint[k])
        for minterm in range(len(reduced_minterms)):
            matrix = np.copy(W[k])
            c=0
            for i in np.where(W[k]==1)[0]:
                matrix[i] = reduced_minterms[minterm][c]
                c+=1
            matrix = np.where(matrix==0,-1,matrix)
            matrix = np.nan_to_num(matrix)
            #matrix = matrix.astype(int)
            matrix_k.append(matrix)
        matrices.append(matrix_k)
    return matrices



In [176]:
W_matrices = create_w_matrices(W)
W_matrices

[[array([ 0., -1.,  0., -1., -1., -1.,  0.,  1.,  0.]),
  array([ 0., -1.,  0., -1., -1.,  1.,  0., -1.,  0.]),
  array([ 0., -1.,  0., -1.,  1., -1.,  0., -1.,  0.]),
  array([ 0., -1.,  0., -1.,  1., -1.,  0.,  1.,  0.]),
  array([ 0., -1.,  0., -1.,  1.,  1.,  0., -1.,  0.]),
  array([ 0., -1.,  0., -1.,  1.,  1.,  0.,  1.,  0.]),
  array([ 0., -1.,  0.,  1., -1., -1.,  0., -1.,  0.]),
  array([ 0., -1.,  0.,  1., -1., -1.,  0.,  1.,  0.]),
  array([ 0., -1.,  0.,  1., -1.,  1.,  0., -1.,  0.]),
  array([ 0., -1.,  0.,  1.,  1., -1.,  0.,  1.,  0.]),
  array([ 0.,  1.,  0., -1., -1.,  1.,  0.,  1.,  0.]),
  array([ 0.,  1.,  0., -1.,  1., -1.,  0.,  1.,  0.]),
  array([ 0.,  1.,  0., -1.,  1.,  1.,  0., -1.,  0.]),
  array([ 0.,  1.,  0.,  1., -1., -1.,  0.,  1.,  0.]),
  array([ 0.,  1.,  0.,  1., -1.,  1.,  0., -1.,  0.]),
  array([ 0.,  1.,  0.,  1., -1.,  1.,  0.,  1.,  0.]),
  array([ 0.,  1.,  0.,  1.,  1., -1.,  0., -1.,  0.]),
  array([ 0.,  1.,  0.,  1.,  1.,  1.,  0., -1.,

In [357]:
increase = int(round(wlen/2-0.1,0))

In [358]:
increase

1

In [133]:
def convert_binary(img):
    (T, img_bin) = cv2.threshold(img, 100, 255, cv2.THRESH_BINARY)
    img_bin[(img_bin==0)]=1
    img_bin[(img_bin==255)]=-1
    img_bin = img_bin.astype(int)
    return img_bin

In [134]:
img_type = 'test'
for img in range(1,5):
    print(f'entrada_{img_type}{img:02d}.jpg')

entrada_test01.jpg
entrada_test02.jpg
entrada_test03.jpg
entrada_test04.jpg


In [185]:

def _convert_binary(img):
    (_, img_bin) = cv2.threshold(img, 100, 255, cv2.THRESH_BINARY)
    img_bin = img_bin.astype('float64')
    img_bin[(img_bin==0)]=1
    img_bin[(img_bin==255)]=-1
    #img_bin = img_bin.astype(int)
    #img_bin[(img_bin==0)]=-1
    return img_bin

def get_images(img_size, img_type):
    ximg = []
    yimg = []
    for img in range(1,img_size+1):
        x = cv2.imread(f'./x/{img_type}{img:02d}.jpg', cv2.IMREAD_GRAYSCALE)
        y = cv2.imread(f'./y/{img_type}{img:02d}.jpg', cv2.IMREAD_GRAYSCALE)
        ximg.append(_convert_binary(x))
        yimg.append(_convert_binary(y))
    return ximg, yimg

In [186]:
get_images(test_size, 'test')

([array([[-1., -1., -1., ..., -1., -1., -1.],
         [-1., -1., -1., ..., -1., -1., -1.],
         [-1., -1., -1., ..., -1., -1., -1.],
         ...,
         [-1., -1., -1., ..., -1., -1., -1.],
         [-1., -1., -1., ..., -1., -1., -1.],
         [-1., -1., -1., ..., -1., -1., -1.]]),
  array([[-1., -1., -1., ..., -1., -1., -1.],
         [-1., -1., -1., ..., -1., -1., -1.],
         [-1., -1., -1., ..., -1., -1., -1.],
         ...,
         [-1., -1., -1., ..., -1., -1., -1.],
         [-1., -1., -1., ..., -1., -1., -1.],
         [-1., -1., -1., ..., -1., -1., -1.]]),
  array([[-1., -1., -1., ..., -1., -1., -1.],
         [-1., -1., -1., ..., -1., -1., -1.],
         [-1., -1., -1., ..., -1., -1., -1.],
         ...,
         [-1., -1., -1., ..., -1., -1., -1.],
         [-1., -1., -1., ..., -1., -1., -1.],
         [-1., -1., -1., ..., -1., -1., -1.]]),
  array([[-1., -1., -1., ..., -1., -1., -1.],
         [-1., -1., -1., ...,  1., -1., -1.],
         [-1., -1., -1., ..., -1

In [187]:
train, ytrain = get_images(train_size, 'train')
val, yval = get_images(val_size, 'val')
test, ytest = get_images(test_size, 'test')

In [188]:
train[0].shape, ytrain[0].shape, val[0].shape, yval[0].shape, test[0].shape, ytest[0].shape

((56, 56), (56, 56), (56, 56), (56, 56), (56, 56), (56, 56))

In [189]:
train[0].max(),train[0].min()

(1.0, -1.0)

In [283]:
def apply_window_(x, W_n, j_n):
    Xl = np.c_[np.zeros([x.shape[0], increase], dtype=int), x, np.zeros([x.shape[0], increase], dtype=int)]
    Xl = np.r_[np.zeros([increase, Xl.shape[1]], dtype=int), Xl, np.zeros([increase, Xl.shape[1]], dtype=int)]
    
    z = np.zeros([x.shape[0], x.shape[0]], dtype=int)
    
    for i in range(z.shape[0]):
        for j in range(z.shape[1]):
            p = Xl[i:i+wlen, j:j+wlen].flatten()
            p = p * W_n
            p = (p[~np.isnan(p)].astype(int))
            p = ''.join(p.astype(str))
            
            indices = np.where(j_n[:, 0] == p)
            if indices[0].size > 0 and j_n[indices[0], 1] == '1':
                z[i, j] = 1
                
    return z

In [191]:
def save_results(Wtrain, Wval, Wtest):
    for img in range(len(Wtrain)):
        s = str(img+1)
        s = s.zfill(2)
        x = copy.deepcopy(Wtrain[img][1])
        x[(x==0)]=255
        x[(x==1)]=0
        cv2.imwrite('./results/train'+s+'.jpg', x)

    for img in range(len(Wval)):
        s = str(img+1)
        s = s.zfill(2)
        x = copy.deepcopy(Wval[img][1])
        x[(x==0)]=255
        x[(x==1)]=0
        cv2.imwrite('./results/val'+s+'.jpg', x)
        
    for img in range(len(Wtest)):
        s = str(img+1)
        s = s.zfill(2)
        x = copy.deepcopy(Wtest[img][1])
        x[(x==0)]=255
        x[(x==1)]=0
        cv2.imwrite('./results/test'+s+'.jpg', x)
    

In [192]:
def save_results_complet(Wtrain, Wval, Wtest):
    for img in range(len(Wtrain)):
        s = str(img+1)
        s = s.zfill(2)
        x = copy.deepcopy(Wtrain[img][0])
        x[(x==0)]=255
        x[(x==1)]=0
        cv2.imwrite('./results_complet/train_op1_'+s+'.jpg', x)
        x = copy.deepcopy(Wtrain[img][1])
        x[(x==0)]=255
        x[(x==1)]=0
        cv2.imwrite('./results_complet/train_op2_'+s+'.jpg', x)

    for img in range(len(Wval)):
        s = str(img+1)
        s = s.zfill(2)
        x = copy.deepcopy(Wval[img][0])
        x[(x==0)]=255
        x[(x==1)]=0
        cv2.imwrite('./results_complet/val_op1_'+s+'.jpg', x)
        x = copy.deepcopy(Wval[img][1])
        x[(x==0)]=255
        x[(x==1)]=0
        cv2.imwrite('./results_complet/val_op2_'+s+'.jpg', x)
        
    for img in range(len(Wtest)):
        s = str(img+1)
        s = s.zfill(2)
        x = copy.deepcopy(Wtest[img][0])
        x[(x==0)]=255
        x[(x==1)]=0
        cv2.imwrite('./results_complet/test_op1_'+s+'.jpg', x)
        x = copy.deepcopy(Wtest[img][1])
        x[(x==0)]=255
        x[(x==1)]=0
        cv2.imwrite('./results_complet/test_op2_'+s+'.jpg', x)
    

In [193]:
def apply_window(x, W_n, j_n):
    Xl = np.c_[np.zeros([x.shape[0], increase], dtype=int), x, np.zeros([x.shape[0], increase], dtype=int)]
    Xl = np.r_[np.zeros([increase, Xl.shape[1]], dtype=int), Xl, np.zeros([increase, Xl.shape[1]], dtype=int)]
    
    z = np.zeros([x.shape[0], x.shape[0]], dtype=int)
    
    for i in range(z.shape[0]):
        for j in range(z.shape[1]):
            p = Xl[i:i+wlen, j:j+wlen].flatten()
            p = p * W_n
            p = (p[~np.isnan(p)].astype(int))
            p = ''.join(p.astype(str))
            
            indices = np.where(j_n[:, 0] == p)
            if indices[0].size > 0 and j_n[indices[0], 1] == '1':
                z[i, j] = 1
                
    return z

In [194]:
def run_window_hood(sample, sample_size, W_current, joint_current, Wlast, layer):
    Wsample = []
    for k in range(sample_size):
        Wsample_k = [] 
        for i in range(nlayer):
            if layer > i:
                Wsample_k.append(Wlast[k][i])
            elif i==0:
                Wsample_k.append(apply_window(sample[k], W_current[i], joint_current[i]))
            else:
                Wsample_k.append(apply_window(Wsample_k[i-1], W_current[i], joint_current[i]))
        Wsample.append(Wsample_k)
    return Wsample

In [216]:
train[0]-1

array([[-2., -2., -2., ..., -2., -2., -2.],
       [-2., -2., -2., ..., -2., -2., -2.],
       [-2., -2., -2., ..., -2., -2., -2.],
       ...,
       [-2., -2., -2., ..., -2., -2., -2.],
       [-2., -2., -2., ..., -2., -2., -2.],
       [-2., -2., -2., ..., -2., -2., -2.]])

In [253]:
train[0].shape

(56, 56)

In [258]:
def apply_convolve(img, W_matrices, bias):
    img_c = np.zeros([img.shape[0], img.shape[1]], dtype=float)
    for kernel in W_matrices:
        img_c += cv2.filter2D(img, -1, kernel)
        img_c = img_c-bias
        img_c[img_c >= 0] = 1
        img_c[img_c < 0] = -1
    return img

In [270]:
my_array = np.array([-1, 2, -3, 4, -5])

# Transform positive values to 1 and negative values to 0
result = (my_array > 0).astype(int)
result

array([0, 1, 0, 1, 0])

In [274]:
(my_array > 0).astype(float)

array([0., 1., 0., 1., 0.])

In [247]:
def run_window_convolve(sample, sample_size, W_matrices, Wlast, layer, bias):
    Wsample = []
    for k in range(sample_size):
        Wsample_k = [] 
        for i in range(nlayer):
            if layer > i:
                Wsample_k.append(Wlast[k][i])
            elif i==0:
                Wsample_k.append(apply_convolve(sample[k], W_matrices[i], bias[i]))
            else:
                Wsample_k.append(apply_convolve(Wsample_k[i-1], W_matrices[i], bias[i]))
        Wsample.append(Wsample_k)
    return Wsample

In [245]:
len(W_matrices[0])

18

In [269]:
bias = np.nansum(W, axis=1) - 1
bias

array([4., 4.])

In [224]:
bias = []
for i in range(nlayer):
    bias.append(np.nansum(W[i])-1)
bias

[4.0, 4.0]

In [260]:
Wtrain = run_window_convolve(train, train_size, W_matrices, W,0, bias)

In [261]:
train[0]

array([[-1., -1., -1., ..., -1., -1., -1.],
       [-1., -1., -1., ..., -1., -1., -1.],
       [-1., -1., -1., ..., -1., -1., -1.],
       ...,
       [-1., -1., -1., ..., -1., -1., -1.],
       [-1., -1., -1., ..., -1., -1., -1.],
       [-1., -1., -1., ..., -1., -1., -1.]])

In [262]:
unique_values, counts = np.unique(Wtrain[0][0], return_counts=True)

# Create a dictionary to store the value-count pairs
value_count_dict = dict(zip(unique_values, counts))

# Print the value-count pairs
for value, count in value_count_dict.items():
    print(f"Value {value} appears {count} times.")

Value -1.0 appears 2754 times.
Value 1.0 appears 382 times.


In [266]:
calculate_error(ytrain, Wtrain)/2

0.18440688775510194

In [267]:
def calculate_error(y, h, et = 'mae'):
    error = 0
    n_samples = len(y)
    for k in range(n_samples):
        if et == 'mae':
            sample_error = np.abs(h[k][-1] - y[k]).sum()
            error += sample_error / (y[k].size)
        elif et== 'iou':
            union = np.sum(np.maximum(h[k][-1],y[k])==1)
            interc = np.sum(h[k][-1] +y[k] == 2)
            error += (1 - interc/union)
    return (error/n_samples)  

In [268]:
calculate_error(ytrain, Wtrain, 'iou')

0.8680290803569619

In [ ]:
train = []
ytrain = []
for img in range(1,train_size+1):
    x = cv2.imread(f'./x/train{img:02d}.jpg', cv2.IMREAD_GRAYSCALE)
    y = cv2.imread(f'./y/train{img:02d}.jpg', cv2.IMREAD_GRAYSCALE)
    train.append(convert_binary(x))
    ytrain.append(convert_binary(y))

In [242]:
len(Wtrain), Wtrain[0][1].shape

(30, (56, 56))

In [243]:
np.unique(Wtrain)

array([-1.])

In [236]:
Wtrain = run_window_hood(train, train_size, W, joint, W, 0)

In [238]:
Wtrain[0][0]

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [17]:
Wval = run_window_hood(val, val_size, W, joint, W, 0)

In [23]:
Wtest = run_window_hood(test, test_size, W, joint, W, 0)

In [264]:
def calculate_error(y, h):
    error = 0
    n_samples = len(y)
    for k in range(n_samples):
        sample_error = np.abs(h[k][-1] - y[k]).sum()
        error += sample_error / (y[k].size * n_samples)
    return error     

In [25]:
def joint_history(joint, nlayer):
    for k in range(nlayer):
        if k==0:
        #joint_temp.append(joint[k][:,1])
            joint_hist = ''.join(joint[k][:,1])
        else:
            joint_hist = joint_hist+''.join(joint[k][:,1])
    return joint_hist


In [26]:
def window_history(W, nlayer, wsize):
    for k in range(nlayer):
        if k==0:
            window_hist = ''.join([''.join(item) for item in np.reshape(W[k], (wsize,)).astype(str)])
        else:
            window_hist = window_hist+''.join([''.join(item) for item in np.reshape(W[k], (wsize,)).astype(str)])
    return window_hist

In [27]:
def get_error_window(W, joint, nlayer, train, train_size, Wtrain, ytrain):
    start = time()
    keep = True
    w_error = calculate_error(ytrain, Wtrain)
    joint_hist = []
    print('----------')
    print('Entrando no get error window')
    print('start error: ', w_error)
    qq = 0
    flg2 = 0
    while keep:
        qq += 1
        #print('Teste ', qq)
        flg = 0
        joint_hist.append(joint_history(joint, nlayer))
        for k in range(nlayer):
            for i in range(len(joint[k])):
                # print('testing layer ', k)
                # print('joint ', joint[k][i][0])
                joint_temp = copy.deepcopy(joint)
                if joint[k][i][1] == '1':
                    joint_temp[k][i][1] = '0'
                else:
                    joint_temp[k][i][1] = '1'
                j_temp = joint_history(joint_temp, nlayer)
                if j_temp not in joint_hist:
                    joint_hist.append(j_temp)
                    W_hood = run_window_hood(train, train_size, W, joint_temp, Wtrain, k)
                    error_hood = calculate_error(ytrain, W_hood)
                    # print('error: ', error_hood)
                    if error_hood < w_error:
                        # print('erro menor')
                        w_error = error_hood
                        # print(w_error)
                        joint_new = copy.deepcopy(joint_temp)
                        Wtrain_new = copy.deepcopy(W_hood)
                        flg = 1
                        flg2 = 1
                    # print('----------')
        if flg == 0:
            #print('Sem erros Menores no teste ', qq)
            keep = False
        else:
            joint = copy.deepcopy(joint_new)
            Wtrain = copy.deepcopy(Wtrain_new)
            #print('error do teste ', qq, ': ', w_error)
            # print('flg = 1 do teste ', qq)

    #print('----------')
    print('end of testing, fl2 = ', flg2)

    if flg2 == 1:
        Wval = run_window_hood(val, val_size, W, joint, W, 0)
    error_val = calculate_error(yval, Wval)
    error = np.array([w_error, error_val])

    #print('-.-.-.-.-.-.-.-.-.-')
    #end = time()
    #print('tempo de execução: {}'.format(end - start))
    save_results(Wtrain, Wval)

    return (joint, error, flg2, Wtrain, Wval)

In [122]:
joint, error, flg, Wtrain, Wval = get_error_window(W, joint, nlayer,train, train_size, Wtrain, ytrain)

----------
Entrando no get error window
start error:  0.14903273809523812
end of testing, fl2 =  1


In [123]:
error

array([0.01852679, 0.01817602])

In [28]:
def save_window(joint, W):

    filename_joint = 'joint.txt'
    pickle.dump(joint, open(filename_joint, 'wb'))

    filename_W = 'W.txt'
    pickle.dump(W, open(filename_W, 'wb'))

In [130]:
save_window(joint, W)

In [29]:
def check_great_neighboors(W, nlayer, wsize, wlen, joint, train, train_size, Wtrain, ytrain, error):
    global w_hist
    flg = 0
    print('\n check_great_neighboors Start')
    
    for k in range(nlayer):
        #print('testing layer ', k)
        nan_idx = np.where(np.isnan(W[k]))[0]
        w_line_temp_base = W[k].copy()
        
        for i in nan_idx:
            W_line_temp = w_line_temp_base.copy()
            W_line_temp[i] = 1
            W_line_temp_NN = W_line_temp.copy()
            W_line_temp_NN[np.isnan(W_line_temp_NN)] = 0
            W_line_temp_NN = W_line_temp_NN.astype(int)
                
            if ''.join(W_line_temp_NN.astype(str)) in windows_continuos:
                #print('W in windows continuos')
                W_temp = W.copy()
                W_temp[k] = W_line_temp
                W_h = window_history(W_temp, nlayer, wsize)

                if W_h not in w_hist['W']:
                    joint_temp = joint.copy()
                    joint_temp[k] = create_joint(W_temp[k])

                    W_train_temp = run_window_hood(train, train_size, W_temp, joint_temp, Wtrain, k)

                    joint_temp, w_error, flg2, W_train_temp, W_val_temp = get_error_window(W_temp, joint_temp, nlayer,train, train_size, W_train_temp, ytrain)
                    #print('new Validation error: ', w_error[1])
                    if w_error[1] < error[1]:
                        #print('erro menor')
                        error_new = w_error
                        joint_new = joint_temp.copy()
                        Wtrain_new = W_train_temp.copy()
                        Wval_new = W_val_temp.copy()
                        W_new = W_temp.copy()
                        save_window(joint_new, W_new)
                        flg = 1
                    #print('----------')

                    w_hist["W"].append(W_h)
                    w_hist["error"].append(w_error)
                    pickle.dump(w_hist, open('W_hist.txt', 'wb'))
            #else:
                #print('W not in window continuos')
    print('flg = ',flg)
    if flg == 1:
        return W_new, joint_new, Wtrain_new, Wval_new, error, flg
    else:
        return W, joint, Wtrain, Wval, error, flg

In [30]:
def check_lesser_neighboors(W, nlayer, wsize, wlen, joint, train, train_size, Wtrain, ytrain, error):
    global w_hist
    flg = 0
    print('\n check_lesser_neighboors Start')
    
    
    for k in range(nlayer):
        #print('testing layer ', k)
        nan_idx = np.where(W[k]==1)[0]
        w_line_temp_base = W[k].copy()
        
        for i in nan_idx:
            W_line_temp = w_line_temp_base.copy()
            W_line_temp[i] = np.nan
            W_line_temp_NN = W_line_temp.copy()
            W_line_temp_NN[np.isnan(W_line_temp_NN)] = 0
            W_line_temp_NN = W_line_temp_NN.astype(int)
            
            if ''.join(W_line_temp_NN.astype(str)) in windows_continuos:
                #print('W in windows continuos')
                W_temp = W.copy()
                W_temp[k] = W_line_temp
                W_h = window_history(W_temp, nlayer, wsize)
                
                if W_h not in w_hist['W']:
                    joint_temp = joint.copy()
                    joint_temp[k] = create_joint(W_temp[k])
                    W_train_temp = run_window_hood(train, train_size, W_temp, joint_temp, Wtrain, k)
                    
                    joint_temp, w_error, flg2, W_train_temp, W_val_temp = get_error_window(W_temp, joint_temp, nlayer,train, train_size, W_train_temp, ytrain)
                    #print('new Validation error: ', w_error[1])
                    if w_error[1] < error[1]:
                        print('error menor')
                        error[0] = w_error[0]
                        error[1] = w_error[1]
                        joint_new = joint_temp.copy()
                        Wtrain_new = W_train_temp.copy()
                        Wval_new = W_val_temp.copy()
                        W_new = W_temp.copy()
                        print('W_new: ', W_new)
                        save_window(joint_new, W_new)
                        flg = 1
                    #print('----------')
                    
                    w_hist["W"].append(W_h)
                    w_hist["error"].append(w_error)
                    pickle.dump(w_hist, open('W_hist.txt', 'wb'))
                #else:
                    #print('W already in history')
            #else:
                #print('W not in window continuos')
    print('flg = ',flg)
    if flg == 1:
        return W_new, joint_new, Wtrain_new, Wval_new, error, flg
    else:
        return W, joint, Wtrain, Wval, error, flg

In [28]:
start = time()
keep = True
print('start Validation error: ', error[1])
print('----------')
qq=0
w_hist = {"W":[],"error":[]}
w_hist["W"].append(window_history(W, nlayer, wsize))
w_hist["error"].append(error)
    
pickle.dump(w_hist, open('W_hist.txt', 'wb'))

while keep:
#for i in range(len(windows_continuos)):
    qq+=1
    print('Teste de Janela ',qq)
    
    W, joint, Wtrain, Wval, error, flg1 = check_lesser_neighboors(W, nlayer, wsize, wlen, joint, train, train_size, Wtrain, ytrain, error)
    W, joint, Wtrain, Wval, error, flg2 = check_great_neighboors(W, nlayer, wsize, wlen, joint, train, train_size, Wtrain, ytrain, error)
    print(error)
    
    if flg1 ==0 & flg2 ==0 :
        keep = False
    else:
        save_window(joint, W)
        save_results(Wtrain, Wval)
        
    if qq>len(windows_continuos):
        keep = False
    
print('end of testing, fl2 = ', flg2)
end = time()
print('tempo de execução: {}'.format(end - start))            


start Validation error:  0.018176020408163265
----------
Teste de Janela  1

 check_lesser_neighboors Start
----------
Entrando no get error window
start error:  0.13271683673469387
end of testing, fl2 =  1
----------
Entrando no get error window
start error:  0.12164115646258505
end of testing, fl2 =  1
----------
Entrando no get error window
start error:  0.11312712585034013
end of testing, fl2 =  1
error menor
W_new:  [array([nan,  1., nan,  1.,  1., nan, nan,  1., nan]), array([nan,  1., nan,  1.,  1.,  1., nan,  1., nan])]
----------
Entrando no get error window
start error:  0.1110863095238095
end of testing, fl2 =  1
error menor
W_new:  [array([nan,  1., nan,  1.,  1.,  1., nan, nan, nan]), array([nan,  1., nan,  1.,  1.,  1., nan,  1., nan])]
----------
Entrando no get error window
start error:  0.08596938775510207
end of testing, fl2 =  1
----------
Entrando no get error window
start error:  0.0858843537414966
end of testing, fl2 =  1
----------
Entrando no get error window
st

In [29]:
save_results(Wtrain, Wval, Wtest)

In [34]:
save_results_complet(Wtrain, Wval, Wtest)

In [36]:
error

array([0.01743197, 0.01702806])

In [33]:
calculate_error(ytest, Wtest)

0.01728316326530612

In [35]:
W

[array([nan,  1., nan,  1.,  1.,  1., nan, nan, nan]),
 array([nan,  1., nan,  1.,  1.,  1.,  1.,  1.,  1.])]

In [41]:
len(w_hist['W'])

32